<a href="https://colab.research.google.com/github/skfo763/Google-ML-Bootcamp-phase1/blob/main/week3/Planar_data_classification_with_onehidden_layer_v6c.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 하나의 은닉층을 활용한 평면 데이터 분류

3주차 프로그래밍 과제에 오신 것을 환영합니다. 이번 과제에서는 여러분들은 처음으로, 1개의 은닉층을 가진 인공 신경망을 만들게 됩니다. 또 이 모델이 지난 2주차 과제에서 개발했던 로지스틱 회귀 모델과 비교했을 때 큰 차이점이 있다는 것도 알게 될 것입니다.

**아래의 것들을 배웁니다** :
- 한 개의 은닉층으로 주어진 데이터셋을 2 개의 서로 다른 클래스로 분류하는 인공 신경망을 만듭니다.
- tanh와 같은 비선형 활성화 함수를 사용합니다.
- 크로스 엔트로피 손실 함수를 계산합니다.
- 순전파, 역전파를 구현합니다.

## 1. 패키지 추가##
본 과제를 수행하는 동안 필요한 파이썬 패키지를 다운로드 받아봅시다.

- [numpy](www.numpy.org)는 파이썬의 가장 기본적인 과학/수학 연산 관련 패키지입니다.
- [sklearn](http://scikit-learn.org/stable/)은 데이터 마이닝, 데이터 분석을 위한 간단하고 효율적인 도구를 제공해줍니다.
- [matplotlib](http://matplotlib.org)은 파이썬 환경에서 그래프를 그릴 수 있도록 해주는 유명한 라이브러리입니다.
- `testCases` 는 과제에서 구현한 함수가 잘 작동하는지 테스트할 수 있는 예시들을 제공합니다.
- `planar_utils` 는 이 과제를 수행하는동안 필요한 다양한 함수들을 제공합니다.

In [ ]:
# Package imports
import numpy as np
import matplotlib.pyplot as plt
from testCases_v2 import *
import sklearn
import sklearn.datasets
import sklearn.linear_model
from planar_utils import plot_decision_boundary, sigmoid, load_planar_dataset, load_extra_datasets

%matplotlib inline

np.random.seed(1) # set a seed so that the results are consistent

## 2. 데이터 셋 ##
우선, 과제에 필요한 데이터셋을 다운로드해봅시다. 아래 코드를 실행하면 "꽃" 모양의 2 클래스 데이터셋 X, Y를 얻을 수 있습니다.

In [ ]:
X, Y = load_planar_dataset()

`matplotlib` 을 사용해 데이터셋을 시각화해봅시다. 데이터는 빨간 점(라벨 y가 0)과, 파란 점(라벨 y가 1)으로 이루어진 꽃 형상을 하고 있습니다. 이 과제의 목표는 이 데이터에 적합한 모델을 만드는 것입니다. 다시 말해, 우리는 이 데이터 중 빨간 점의 영역과 파란 점의 영역을 구분하는 모델을 만들고자 합니다.

In [ ]:
# Visualize the data:
plt.scatter(X[0, :], X[1, :], c=Y, s=40, cmap=plt.cm.Spectral);

이 데이터 셋은
- 각 데이터의 특성 (x1, x2) 으로 구성된 numpy-array (matrix) X
- 라벨링된 결과값 Y (red: 0, blue: 1)로 구성된 numpy array (vector) Y

로 이루어져 있습니다.

이 데이터가 어떻게 생겼는지 좀 더 자세히 알아봅시다.

**연습 문제**: 훈련 세트의 개수는 얼마나 될까요? 추가로, X, Y 변수의 shape는 무엇일까요?
**힌트**: numpy array의 shape를 얻는 방법은 무엇일까요?([help](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.shape.html))


In [ ]:
### START CODE HERE ### (≈ 3 lines of code)
shape_X = None
shape_Y = None
m = None  # training set size
### END CODE HERE ###

print ('The shape of X is: ' + str(shape_X))
print ('The shape of Y is: ' + str(shape_Y))
print ('I have m = %d training examples!' % (m))

**모범 답안**:
       
<table style="width:20%">
  <tr>
    <td>shape of X</td>
    <td> (2, 400) </td> 
  </tr>
  <tr>
    <td>shape of Y</td>
    <td>(1, 400) </td> 
  </tr>
    <tr>
    <td>m</td>
    <td> 400 </td> 
  </tr>
</table>

## 3. 단일 로지스틱 회귀 ##

전체 인공 신경망을 구현하기 이전에, 이 문제에 적합한 로지스틱 회귀 모델이 어떻게 동작하는지부터 확인해 보겠습니다. 저번 과제처럼 직접 구현하기보단, `sklearn` 라이브러리에서 미리 구현된 빌트인 함수를 사용하겠습니다. 아래 코드 블록을 실행시켜 주어진 데이터셋에 대한 로지스틱 회귀 분류기를 훈련시켜 봅시다.

In [ ]:
# Train the logistic regression classifier
clf = sklearn.linear_model.LogisticRegressionCV();
clf.fit(X.T, Y.T);

이제 여러분은 이 모델에 대한 decision boundary를 그릴 수 있습니다. 아래 코드 블록을 실행시켜 보세요.

In [ ]:
# Plot the decision boundary for logistic regression
plot_decision_boundary(lambda x: clf.predict(x), X, Y)
plt.title("Logistic Regression")

# Print accuracy
LR_predictions = clf.predict(X.T)
print ('Accuracy of logistic regression: %d ' % float((np.dot(Y,LR_predictions) + np.dot(1-Y,1-LR_predictions))/float(Y.size)*100) +
       '% ' + "(percentage of correctly labelled datapoints)")

**모범 답안**:

<table style="width:20%">
  <tr>
    <td>Accuracy</td>
    <td> 47% </td> 
  </tr>
</table>

**해석**: 주어진 데이터셋 을 선형 함수로 분리할 수 없으므로, 로지스틱 회귀가 재대로 수행되지 않습니다. 인공신경망이 더 잘 작동할 수 있도록 이번 강의에서 배웠던 내용을 적용해봅시다.

## 4. 인공 신경망 모델 ##

로지스틱 회귀는 주어진 꽃 형상의 데이터셋에는 잘 동작하지 않는것을 확인할 수 있었습니다. 이제 우리는 하나의 은닉층을 가지는 인공 신경망을 훈련시켜 볼 것입니다.

**훈련시킬 모델 구조도** :
![model_overview]()

**모델의 수학적 구성** :

한개의 데이터 $x^{(i)}$에 대하여,

$$z^{[1] (i)} =  W^{[1]} x^{(i)} + b^{[1]}\tag{1}$$ 
$$a^{[1] (i)} = \tanh(z^{[1] (i)})\tag{2}$$
$$z^{[2] (i)} = W^{[2]} a^{[1] (i)} + b^{[2]}\tag{3}$$
$$\hat{y}^{(i)} = a^{[2] (i)} = \sigma(z^{ [2] (i)})\tag{4}$$
$$y^{(i)}_{prediction} = \begin{cases} 1 & \mbox{if } a^{[2](i)} > 0.5 \\ 0 & \mbox{otherwise } \end{cases}\tag{5}$$

모든 데이터 셋에 대하여 예측값을 계산한 이후, 아래의 정의대로 비용 함수 J를 계산할 수 있습니다:

$$J = - \frac{1}{m} \sum\limits_{i = 0}^{m} \large\left(\small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right)  \large  \right) \small \tag{6}$$

**리마인더** :
1. 인공 신경망의 구조를 정의합니다 (입력 층의 개수, 은닉층의 개수, 등등..)
2. 모델의 파라미터들을 초기화합니다.
3. 아래의 동작을 반복합니다.
  - 정방향 순전파
  - 손실함수 계산
  - 오차 역전파
  - 파라미터 업데이트 (경사 하강법)

1-3 단계를 계산하는 도우미 함수를 구현한 다음 `nn_model()` 이라고하는 하나의 함수로 통합하는 경우가 많습니다. `nn_model()` 을 구현하고 올바른 매개 변수를 학습 한 후에는 새 데이터에 대해 예측할 수 있습니다.

### 4-1. 인공 신경망 구조 정의하기 ###

**연습 문제**: 아래 세 가지 변수를 정의하시오:
- `n_x` : 입력 층의 개수
- `n_h` : 은닉 층의 개수
- `n_y` : 출력 층의 개수

**힌트**: 앞서 계산했던 데이터셋 X와 Y의 shape를 이용해서 n_x와 n_y를 찾아보세요, 물론 은닉 층의 개수 n_h는 4로 고정되어야 할 것입니다.

In [ ]:
# GRADED FUNCTION: layer_sizes

def layer_sizes(X, Y):
    """
    Arguments:
    X -- input dataset of shape (input size, number of examples)
    Y -- labels of shape (output size, number of examples)
    
    Returns:
    n_x -- the size of the input layer
    n_h -- the size of the hidden layer
    n_y -- the size of the output layer
    """
    ### START CODE HERE ### (≈ 3 lines of code)
    n_x = None # size of input layer
    n_h = None
    n_y = None # size of output layer
    ### END CODE HERE ###
    return (n_x, n_h, n_y)

In [ ]:
X_assess, Y_assess = layer_sizes_test_case()
(n_x, n_h, n_y) = layer_sizes(X_assess, Y_assess)
print("The size of the input layer is: n_x = " + str(n_x))
print("The size of the hidden layer is: n_h = " + str(n_h))
print("The size of the output layer is: n_y = " + str(n_y))

**모범 답안** (이 데이터는 실제로 인공 신경망에 사용될 데이터는 아닙니다).

<table style="width:20%">
  <tr>
    <td>n_x</td>
    <td> 5 </td> 
  </tr>
    <tr>
    <td>n_h</td>
    <td> 4 </td> 
  </tr>
    <tr>
    <td>n_y</td>
    <td> 2 </td> 
  </tr>
</table>

### 4-2. 모델의 파라미터 초기화하기 ###

**연습 문제**: `intialize_parameters()` 를 구현해봅시다.

**지시 사항** :
- 파라미터의 사이즈가 올바르게 적용되었는지 확인하세요. 필요하다면 위의 신경망 구조도를 참조해도 좋습니다.
- 각 layer의 가중치를 랜덤 값으로 초기화해야 합니다.
  - `np.random.randn(a, b) * 0.01` 코드를 사용해 (a, b)의 형태를 가지고 있는 행렬을 랜덤으로 초기화할 수 있습니다.
- bias 값을 0으로 초기화합니다.
  - `np.zeros(a, b)` 코드를 사용해 (a, b) 형태의 행렬을 0으로 초기화할 수 있습니다.

In [ ]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters(n_x, n_h, n_y):
    """
    Argument:
    n_x -- size of the input layer
    n_h -- size of the hidden layer
    n_y -- size of the output layer
    
    Returns:
    params -- python dictionary containing your parameters:
                    W1 -- weight matrix of shape (n_h, n_x)
                    b1 -- bias vector of shape (n_h, 1)
                    W2 -- weight matrix of shape (n_y, n_h)
                    b2 -- bias vector of shape (n_y, 1)
    """
    
    np.random.seed(2) # we set up a seed so that your output matches ours although the initialization is random.
    
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = None
    b1 = None
    W2 = None
    b2 = None
    ### END CODE HERE ###
    
    assert (W1.shape == (n_h, n_x))
    assert (b1.shape == (n_h, 1))
    assert (W2.shape == (n_y, n_h))
    assert (b2.shape == (n_y, 1))
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
n_x, n_h, n_y = initialize_parameters_test_case()

parameters = initialize_parameters(n_x, n_h, n_y)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**모범 답안**:

<table style="width:90%">
  <tr>
    <td>W1</td>
    <td> [[-0.00416758 -0.00056267]
 [-0.02136196  0.01640271]
 [-0.01793436 -0.00841747]
 [ 0.00502881 -0.01245288]] </td> 
  </tr>
  
  <tr>
    <td>b1</td>
    <td> [[ 0.]
 [ 0.]
 [ 0.]
 [ 0.]] </td> 
  </tr>
  
  <tr>
    <td>W2</td>
    <td> [[-0.01057952 -0.00909008  0.00551454  0.02292208]]</td> 
  </tr>
  

  <tr>
    <td>b2</td>
    <td> [[ 0.]] </td> 
  </tr>
  
</table>


### 4-3. 반복하기 ###

**연습 문제**: `forward_propagation()` 함수를 구현하세요.

**지시 사항**:
- 위에 나와 있는 수식을 참고하세요.
- 이 주피터 노트북에 빌트인된 `sigmoid()` 함수를 사용할 수 있습니다.
- numpy 라이브러리에 있는 `np.tanh()` 함수를 사용할 수 있습니다.
- 구현 해야할 일련의 순서는
  - 앞서 구현한 `initalize_parameters()` 함수의 리턴값인 `parameter` 인자로부터 모델의 파라미터를 얻으세요.
  - Forward Propagation을 구현하세요. $Z^{[1]}, A^{[1]}, Z^{[2]}$와 $A^{[2]}$(모든 훈련 세트에 대한 예측 결과값을 담은 벡터) 를 계산하세요
- Back Propagation에 사용될 값들은 "cache"에 저장됩니다. cache는 이후 구현할 back propagation 함수의 인자로 들어갈 예정입니다.

In [ ]:
# GRADED FUNCTION: forward_propagation

def forward_propagation(X, parameters):
    """
    Argument:
    X -- input data of size (n_x, m)
    parameters -- python dictionary containing your parameters (output of initialization function)
    
    Returns:
    A2 -- The sigmoid output of the second activation
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2"
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = None
    b1 = None
    W2 = None
    b2 = None
    ### END CODE HERE ###
    
    # Implement Forward Propagation to calculate A2 (probabilities)
    ### START CODE HERE ### (≈ 4 lines of code)
    Z1 = None
    A1 = None
    Z2 = None
    A2 = None
    ### END CODE HERE ###
    
    assert(A2.shape == (1, X.shape[1]))
    
    cache = {"Z1": Z1,
             "A1": A1,
             "Z2": Z2,
             "A2": A2}
    
    return A2, cache

In [ ]:
X_assess, parameters = forward_propagation_test_case()
A2, cache = forward_propagation(X_assess, parameters)

# Note: we use the mean here just to make sure that your output matches ours. 
print(np.mean(cache['Z1']) ,np.mean(cache['A1']),np.mean(cache['Z2']),np.mean(cache['A2']))

**모범 답안**:
<table style="width:50%">
  <tr>
    <td> 0.262818640198 0.091999045227 -1.30766601287 0.212877681719 </td> 
  </tr>
</table>

이제 각 샘플 데이터의 결과값인 $ a ^ {[2] (i)} $들을 모두 포함하는 $ A ^ {[2]} $ (Python 변수 "ʻA2`"에서)를 계산 했으므로, 우리는 다음과 같이 비용 함수를 계산할 수 있습니다. :

$$J = - \frac{1}{m} \sum\limits_{i = 1}^{m} \large{(} \small y^{(i)}\log\left(a^{[2] (i)}\right) + (1-y^{(i)})\log\left(1- a^{[2] (i)}\right) \large{)} \small\tag{13}$$

**연습 문제**: cost $J$ 를 계산하는 `compute_cost()` 함수를 구현하세요

**지시 사항**: 
- 크로스 엔트로피 손실함수를 구현하는데 다양한 방법을 사용할 수 있습니다. 약간의 도움을 주기 위해, 우리가 $- \sum\limits_{i=0}^{m}  y^{(i)}\log(a^{[2](i)})$를 어떻게 구현했는지 알려드리겠습니다:
```python
logprobs = np.multiply(np.log(A2),Y)
cost = - np.sum(logprobs)                # no need to use a for loop!
```

여러분은 `np.multiply()` 함수를 적용한 다음 `np.sum()`을 사용하거나, 혹은 다이렉트로 `np.dot()` 함수를 사용할 수 있습니다. 

`np.multiply()` 다음에 `np.sum()` 을 사용하면 최종 결과는 `float` 유형이 되고, `np.dot()` 을 사용하면 결과는 2차원 numpy array가 됩니다. 

`np.squeeze()`를 사용하여 필요없는 중복된 dimensions를 제거할 수 있습니다. 단일 `float` 값의 경우라면, 0차원 배열로 축소됩니다. `float()` 함수를 이용하면 배열을 `float` 유형으로 캐스팅 할 수도 있습니다.


In [ ]:
# GRADED FUNCTION: compute_cost

def compute_cost(A2, Y, parameters):
    """
    Computes the cross-entropy cost given in equation (13)
    
    Arguments:
    A2 -- The sigmoid output of the second activation, of shape (1, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    parameters -- python dictionary containing your parameters W1, b1, W2 and b2
    [Note that the parameters argument is not used in this function, 
    but the auto-grader currently expects this parameter.
    Future version of this notebook will fix both the notebook 
    and the auto-grader so that `parameters` is not needed.
    For now, please include `parameters` in the function signature,
    and also when invoking this function.]
    
    Returns:
    cost -- cross-entropy cost given equation (13)
    
    """
    
    m = Y.shape[1] # number of example

    # Compute the cross-entropy cost
    ### START CODE HERE ### (≈ 2 lines of code)
    logprobs = None
    cost = None
    ### END CODE HERE ###
    
    cost = float(np.squeeze(cost))  # makes sure cost is the dimension we expect. 
                                    # E.g., turns [[17]] into 17 
    assert(isinstance(cost, float))
    
    return cost

In [ ]:
A2, Y_assess, parameters = compute_cost_test_case()

print("cost = " + str(compute_cost(A2, Y_assess, parameters)))

**모범 답안**:
<table style="width:20%">
  <tr>
    <td>cost</td>
    <td> 0.693058761... </td> 
  </tr>
</table>

앞서 forward propagation을 구현하는 과정에서 값을 저장했던 `cache` 변수를 활용해서 backward propagation을 구현해봅시다.

**연습 문제**: `backward_propagation()` 함수를 구현하세요

**지시 사항**: Backpropagation은 일반적으로 딥 러닝 분야에서 수학적으로 가장 어려운 부분입니다. 당신을 돕기 위해서, 이번 강의에서 사용했던 슬라이드를 첨부하겠습니다. 이번 과제에선 벡터화된 방식으로 모델을 학습하고 있으므로 슬라이드 오른쪽에 있는 6개의 공식을 참조하는 것이 좋을 것입니다.

<img src="images/grad_summary.png" style="width:600px;height:300px;">

<!--
$\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } = \frac{1}{m} (a^{[2](i)} - y^{(i)})$

$\frac{\partial \mathcal{J} }{ \partial W_2 } = \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } a^{[1] (i) T} $

$\frac{\partial \mathcal{J} }{ \partial b_2 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)}}}$

$\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} } =  W_2^T \frac{\partial \mathcal{J} }{ \partial z_{2}^{(i)} } * ( 1 - a^{[1] (i) 2}) $

$\frac{\partial \mathcal{J} }{ \partial W_1 } = \frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)} }  X^T $

$\frac{\partial \mathcal{J} _i }{ \partial b_1 } = \sum_i{\frac{\partial \mathcal{J} }{ \partial z_{1}^{(i)}}}$

- Note that $*$ denotes elementwise multiplication.
- The notation you will use is common in deep learning coding:
    - dW1 = $\frac{\partial \mathcal{J} }{ \partial W_1 }$
    - db1 = $\frac{\partial \mathcal{J} }{ \partial b_1 }$
    - dW2 = $\frac{\partial \mathcal{J} }{ \partial W_2 }$
    - db2 = $\frac{\partial \mathcal{J} }{ \partial b_2 }$
!-->

- 팁:
  - dZ1을 계산하기 위해서, $g^{[1]'}(Z^{[1]})$를 계산해야 합니다. $g^{[1]}(.)$는 할성화 함수이므로, $ a = g ^ {[1]} (z) $이면 $ g ^ {[1] '} (z) = 1- a ^ 2 $입니다. 따라서 $g^{[1]'}(Z^{[1]})$값을 `(1 - np.power(A1, 2))` 코드를 사용해 계산할 수 있습니다


In [ ]:
# GRADED FUNCTION: backward_propagation

def backward_propagation(parameters, cache, X, Y):
    """
    Implement the backward propagation using the instructions above.
    
    Arguments:
    parameters -- python dictionary containing our parameters 
    cache -- a dictionary containing "Z1", "A1", "Z2" and "A2".
    X -- input data of shape (2, number of examples)
    Y -- "true" labels vector of shape (1, number of examples)
    
    Returns:
    grads -- python dictionary containing your gradients with respect to different parameters
    """
    m = X.shape[1]
    
    # First, retrieve W1 and W2 from the dictionary "parameters".
    ### START CODE HERE ### (≈ 2 lines of code)
    W1 = None
    W2 = None
    ### END CODE HERE ###
        
    # Retrieve also A1 and A2 from dictionary "cache".
    ### START CODE HERE ### (≈ 2 lines of code)
    A1 = None
    A2 = None
    ### END CODE HERE ###
    
    # Backward propagation: calculate dW1, db1, dW2, db2. 
    ### START CODE HERE ### (≈ 6 lines of code, corresponding to 6 equations on slide above)
    dZ2 = None
    dW2 = None
    db2 = None
    dZ1 = None
    dW1 = None
    db1 = None
    ### END CODE HERE ###
    
    grads = {"dW1": dW1,
             "db1": db1,
             "dW2": dW2,
             "db2": db2}
    
    return grads

In [ ]:
parameters, cache, X_assess, Y_assess = backward_propagation_test_case()

grads = backward_propagation(parameters, cache, X_assess, Y_assess)
print ("dW1 = "+ str(grads["dW1"]))
print ("db1 = "+ str(grads["db1"]))
print ("dW2 = "+ str(grads["dW2"]))
print ("db2 = "+ str(grads["db2"]))

**모범 답안**:

<table style="width:80%">
  <tr>
    <td>dW1</td>
    <td> [[ 0.00301023 -0.00747267]
 [ 0.00257968 -0.00641288]
 [-0.00156892  0.003893  ]
 [-0.00652037  0.01618243]] </td> 
  </tr>
  <tr>
    <td>db1</td>
    <td>  [[ 0.00176201]
 [ 0.00150995]
 [-0.00091736]
 [-0.00381422]] </td> 
  </tr>
  <tr>
    <td>dW2</td>
    <td> [[ 0.00078841  0.01765429 -0.00084166 -0.01022527]] </td> 
  </tr>
  <tr>
    <td>db2</td>
    <td> [[-0.16655712]] </td> 
  </tr>
  
</table>  

**연습 문제**: 파라미터를 업데이트하는 코드를 구현하세요. 경사 하강법을 사용합니다. 다시 말해 (W1, b1, W2, b2)를 업데이트하려면 (dW1, db1, dW2, db2)를 사용해야 합니다.

**경사 하강법 공식**: $ \theta = \theta - \alpha \frac{\partial J }{ \partial \theta }$ ($\alpha$ = 학습률, $\theta$ = 파라미터)

**Illustration**: 아래 gif 파일은 좋은 학습률 (수렴하는) 을 가진 경사 하강법 알고리즘과 나쁜 학습률 (발산하는)을 가진 경사 하강법 알고리즘을 나타냅니다.

이미지 제공 : Adam Harley

<img src="images/sgd.gif" style="width:400;height:400;">

<img src="images/sgd_bad.gif" style="width:400;height:400;">
  

In [ ]:
# GRADED FUNCTION: update_parameters

def update_parameters(parameters, grads, learning_rate = 1.2):
    """
    Updates parameters using the gradient descent update rule given above
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients 
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
    """
    # Retrieve each parameter from the dictionary "parameters"
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = None
    b1 = None
    W2 = None
    b2 = None
    ### END CODE HERE ###
    
    # Retrieve each gradient from the dictionary "grads"
    ### START CODE HERE ### (≈ 4 lines of code)
    dW1 = None
    db1 = None
    dW2 = None
    db2 = None
    ## END CODE HERE ###
    
    # Update rule for each parameter
    ### START CODE HERE ### (≈ 4 lines of code)
    W1 = None
    b1 = None
    W2 = None
    b2 = None
    ### END CODE HERE ###
    
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    
    return parameters

In [ ]:
parameters, grads = update_parameters_test_case()
parameters = update_parameters(parameters, grads)

print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**모범 답안**:

<table style="width:80%">
  <tr>
    <td>W1</td>
    <td> [[-0.00643025  0.01936718]
 [-0.02410458  0.03978052]
 [-0.01653973 -0.02096177]
 [ 0.01046864 -0.05990141]]</td> 
  </tr>
  <tr>
    <td>b1</td>
    <td> [[ -1.02420756e-06]
 [  1.27373948e-05]
 [  8.32996807e-07]
 [ -3.20136836e-06]]</td> 
  </tr>
  <tr>
    <td>W2</td>
    <td> [[-0.01041081 -0.04463285  0.01758031  0.04747113]] </td> 
  </tr>
  <tr>
    <td>b2</td>
    <td> [[ 0.00010457]] </td> 
  </tr>
</table>  

### 4-4. nn_model() 함수로 일련의 과정을 통합하기 ###

**연습 문제**: 인공신경망 `nn_model()` 함수를 구현하세요

**지시 사항**: 앞서 구현한 함수들을 올바른 순서대로 배치하세요.

In [ ]:
# GRADED FUNCTION: nn_model

def nn_model(X, Y, n_h, num_iterations = 10000, print_cost=False):
    """
    Arguments:
    X -- dataset of shape (2, number of examples)
    Y -- labels of shape (1, number of examples)
    n_h -- size of the hidden layer
    num_iterations -- Number of iterations in gradient descent loop
    print_cost -- if True, print the cost every 1000 iterations
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    np.random.seed(3)
    n_x = layer_sizes(X, Y)[0]
    n_y = layer_sizes(X, Y)[2]
    
    # Initialize parameters
    ### START CODE HERE ### (≈ 1 line of code)
    parameters = None
    ### END CODE HERE ###
    
    # Loop (gradient descent)

    for i in range(0, num_iterations):
         
        ### START CODE HERE ### (≈ 4 lines of code)
        # Forward propagation. Inputs: "X, parameters". Outputs: "A2, cache".
        A2, cache = None
        
        # Cost function. Inputs: "A2, Y, parameters". Outputs: "cost".
        cost = None
 
        # Backpropagation. Inputs: "parameters, cache, X, Y". Outputs: "grads".
        grads = None
 
        # Gradient descent parameter update. Inputs: "parameters, grads". Outputs: "parameters".
        parameters = None
        
        ### END CODE HERE ###
        
        # Print the cost every 1000 iterations
        if print_cost and i % 1000 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))

    return parameters

In [ ]:
X_assess, Y_assess = nn_model_test_case()
parameters = nn_model(X_assess, Y_assess, 4, num_iterations=10000, print_cost=True)
print("W1 = " + str(parameters["W1"]))
print("b1 = " + str(parameters["b1"]))
print("W2 = " + str(parameters["W2"]))
print("b2 = " + str(parameters["b2"]))

**Expected Output**:

<table style="width:90%">
<tr> 
    <td> 
        cost after iteration 0
    </td>
    <td> 
        0.692739
    </td>
</tr>
<tr> 
    <td> 
        <center> $\vdots$ </center>
    </td>
    <td> 
        <center> $\vdots$ </center>
    </td>
</tr>
  <tr>
    <td>W1</td>
    <td> [[-0.65848169  1.21866811]
 [-0.76204273  1.39377573]
 [ 0.5792005  -1.10397703]
 [ 0.76773391 -1.41477129]]</td> 
  </tr>
  
  <tr>
    <td>b1</td>
    <td> [[ 0.287592  ]
 [ 0.3511264 ]
 [-0.2431246 ]
 [-0.35772805]] </td> 
  </tr>
  <tr>
    <td>W2</td>
    <td> [[-2.45566237 -3.27042274  2.00784958  3.36773273]] </td> 
  </tr>
  <tr>
    <td>b2</td>
    <td> [[ 0.20459656]] </td> 
  </tr>
</table>

### 4-5. 예측하기 ###

**연습 문제** : 지금까지 개발한 모델을 바탕으로, 주어진 문제를 예측하는 `predict()` 함수를 만들어봅시다. 예측을 위해서는 forward propagation을 사용하면 됩니다.

**기억할 것**

$y_{prediction} = \mathbb 1 \text{{activation > 0.5}} = \begin{cases}
      1 & \text{if}\ activation > 0.5 \\
      0 & \text{otherwise}
    \end{cases}$

예를 들어, 임계치에 따라 행렬 X의 각 항목을 0과 1로 설정하려면 아래를 수행합니다. ```X_new = (X > threshold)```

In [ ]:
# GRADED FUNCTION: predict

def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    ### START CODE HERE ### (≈ 2 lines of code)
    A2, cache = None
    predictions = None
    ### END CODE HERE ###
    
    return predictions

In [ ]:
parameters, X_assess = predict_test_case()

predictions = predict(parameters, X_assess)
print("predictions mean = " + str(np.mean(predictions)))

**모범 답안**: 
<table style="width:40%">
  <tr>
    <td>predictions mean</td>
    <td> 0.666666666667 </td> 
  </tr>
  
</table>

모델을 실행하고 평면형 데이터 세트에서 어떻게 수행되는지 살펴볼 시간입니다. 다음 코드를 실행하여 $n_h$ 숨겨진 유닛의 단일 숨겨진 레이어로 모델을 테스트하십시오.

In [ ]:
# Build a model with a n_h-dimensional hidden layer
parameters = nn_model(X, Y, n_h = 4, num_iterations = 10000, print_cost=True)

# Plot the decision boundary
plot_decision_boundary(lambda x: predict(parameters, x.T), X, Y)
plt.title("Decision Boundary for hidden layer size " + str(4))

**모범 답안**:
<table style="width:40%">
  <tr>
    <td>Cost after iteration 9000</td>
    <td> 0.218607 </td> 
  </tr>
  
</table>

In [ ]:
# Print accuracy
predictions = predict(parameters, X)
print ('Accuracy: %d' % float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100) + '%')

**모범 답안**: 

<table style="width:15%">
  <tr>
    <td>Accuracy</td>
    <td> 90% </td> 
  </tr>
</table>

로지스틱 회귀에 비해 정확도가 정말 높습니다. 모델은 꽃잎 패턴의 데이터 분포를 학습했습니다. 신경망은 로지스틱 회귀와 달리 매우 비선형적인 decision boundaries 까지 학습 할 수 있습니다.

이제 다양한 은닉 층의 사이즈를 사용해 보겠습니다.

### 4-6. 은닉 층의 크기 조정하기(선택 사항/평가 X) ###

다음 코드를 실행해보세요. 1-2분이 소요될 수 있습니다. 다양한 은닉층 크기에 대해 모델의 다양한 동작을 살펴볼 수 있습니다.



In [ ]:
# This may take about 2 minutes to run

plt.figure(figsize=(16, 32))
hidden_layer_sizes = [1, 2, 3, 4, 5, 20, 50]
for i, n_h in enumerate(hidden_layer_sizes):
    plt.subplot(5, 2, i+1)
    plt.title('Hidden Layer of size %d' % n_h)
    parameters = nn_model(X, Y, n_h, num_iterations = 5000)
    plot_decision_boundary(lambda x: predict(parameters, x.T), X, Y)
    predictions = predict(parameters, X)
    accuracy = float((np.dot(Y,predictions.T) + np.dot(1-Y,1-predictions.T))/float(Y.size)*100)
    print ("Accuracy for {} hidden units: {} %".format(n_h, accuracy))

**해석**:
- 더 큰 모델 (은닉층이 더 많음)은 결국 가장 큰 모델이 데이터를 과적합 할 때까지 훈련 세트에 더 잘 맞을 수 있습니다.
- 가장 좋은 은닉층 크기는 n_h = 5 정도 인 것 같습니다. 실제로 여기 주변의 값은 눈에 띄는 과적 합없이 데이터에 잘 맞는 것처럼 보입니다.
- 나중에 과적 합없이 매우 큰 모델 (예 : n_h = 50)을 사용할 수있는 정규화에 대해서도 배우게됩니다.

**추가 질문**:

참고 : 오른쪽 상단의 파란색 "과제 제출" 버튼을 클릭하여 과제를 제출해야합니다.

원하는 경우 아래의 몇 가지 선택적인 질문들의 답을 찾아보세요

- tanh 활성화를 sigmoid, 혹은 ReLU 활성화 함수로 변경하면 어떻게 될까요?
- learning_rate를 조정해보세요. 무슨 일이 일어날까요?
- 데이터 세트를 변경하면 어떻게 되나요? (아래 5 부 참조!)

**이번 과제에선 아래 내용을 배웠습니다.**

- 은닉층을 사용한 완전한 신경망 구축
- 비선형 unit을 잘 활용할 수 있게 되었습니다.
- 순방향 전파 및 역 전파를 구현
- 은닉 레이어 크기를 다양화할 때의 영향을 알 수 있게 되었습니다.

수고하셨습니다!

## 5) 다른 데이터셋을 활용해보기 ##

원하는 경우 다음 데이터 세트 각각에 대해 전체 학습 과정을 다시 수행할 수 있습니다.

In [ ]:
# Datasets
noisy_circles, noisy_moons, blobs, gaussian_quantiles, no_structure = load_extra_datasets()

datasets = {"noisy_circles": noisy_circles,
            "noisy_moons": noisy_moons,
            "blobs": blobs,
            "gaussian_quantiles": gaussian_quantiles}

### START CODE HERE ### (choose your dataset)
dataset = "noisy_moons"
### END CODE HERE ###

X, Y = datasets[dataset]
X, Y = X.T, Y.reshape(1, Y.shape[0])

# make blobs binary
if dataset == "blobs":
    Y = Y%2

# Visualize the data
plt.scatter(X[0, :], X[1, :], c=Y, s=40, cmap=plt.cm.Spectral);

본 프로그래밍 과제를 완료하신 것을 축하드립니다!

참고 문헌:
- http://scs.ryerson.ca/~aharley/neural-networks/
- http://cs231n.github.io/neural-networks-case-study/